In [1]:
import ee
import rioxarray as rio
import xarray as xr
from osgeo import gdal
from pyproj import CRS
import geemap as gm

In [ ]:

# ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
# Open data with Gdal and get the CRS information
file_path = '/Users/moyanofe/BigData/GeoSpatial/Carboscope/Inversions/s85oc_v2022_daily.nc' 
# Open data with xarray
ds = xr.open_dataset(file_path)

# Locate CRS info in the file manually
cc = ds.attrs

print(cc)


{'title': 'Jena CarboScope results', 'history': 'Inversion code by Christian Roedenbeck', 'institution': 'MPI Biogeochemistry Jena', 'references': 'http://www.BGC-Jena.mpg.de/CarboScope/', 'Conventions': 'CF-1.1', 'species': 'co2', 'yrfi': 1951, 'yrfe': 2022, 'valid_period': '1985-2021 (inclusive)', 'yrfi_valid': 1985, 'yrfe_valid': 2021, 'yrvi': 1985, 'yrve': 2021, 'normalization_period': '1985-2021 (inclusive)', 'yrni': 1985, 'yrne': 2021, 'dirname': 'OUTPUT14.005r8t+tm3_ncep_fg.1951-2022_2Hx2.co2.-S-.s85v2022.ATMv2022_RESIDneeMSC57_RESIDoc.nsigma2', 'filename': 'mu1.0_070_flux_asis+YRVI-YRVE.nc', 'c0': 310.052}
